In [1]:
#Import libraries

import argparse
from path import Path

import torch
import torch.backends.cudnn as cudnn
import torch.nn.functional as F
import models
from tqdm import tqdm

import torchvision.transforms as transforms
import flow_transforms
from imageio import imread, imwrite
import numpy as np
from util import flow2rgb

import matplotlib.pyplot as plt


c:\Users\Alex\Desktop\IMT\MCE\Computer vision\project\FlowNetPytorch\models\util.py:10: ImportWarning: failed to load custom correlation modulewhich is needed for FlowNetC
  warnings.warn("failed to load custom correlation module"


In [2]:
path_to_model = "./pretrained_model/flownets_EPE1.951.pth.tar"

#Load model

def set_cuda():
    # setting the cuda device
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print(device)
    return device


def load_model(model_path, cuda_device):
    # Importing the model
    network_data = torch.load(model_path, map_location=cuda_device)
    print("=> using pre-trained model '{}'".format(network_data['arch']))
    model = models.__dict__[network_data['arch']](network_data).to(cuda_device)
    model.eval()
    cudnn.benchmark = True

    if 'div_flow' in network_data.keys():
        div_flow = network_data['div_flow']
    else:
        div_flow = 20.0

    return model, div_flow




@torch.no_grad()
def inference(nb_start, nb_end, name_of_the_figure, cuda_device, model, div_flow):
    # create the folder for saving the results
    path_to_save = Path('my_results')
    if not path_to_save.exists():
        path_to_save.mkdir()


    # Data loading code
    input_transform = transforms.Compose([
        flow_transforms.ArrayToTensor(),
        transforms.Normalize(mean=[0,0,0], std=[255,255,255]),
        transforms.Normalize(mean=[0.411,0.432,0.45], std=[1,1,1])
    ])

    ###############importing images and nework form the paths################
    path_to_sequesnces = "./sequences-train/"
    name = name_of_the_figure + '-'

    
    ###############importing images and nework form the paths################


    for i in range(nb_start, nb_end):

        n_img_1 = i
        n_img_2 = i+1

        path_img1 = path_to_sequesnces + name + str(n_img_1).zfill(3) + ".bmp"
        path_img2 = path_to_sequesnces + name + str(n_img_2).zfill(3) + ".bmp"

        img1 = imread(path_img1)
        img2 = imread(path_img2)
        img1 = input_transform(img1)
        img2 = input_transform(img2)

        input_var = torch.cat([img1, img2]).unsqueeze(0)
        input_var = input_var.to(cuda_device)
        output = model(input_var)
        # print(output.shape)

        
        flow_output = output.squeeze(0)
        

        file_name = "./my_results/" + name + str(n_img_1).zfill(3) + "-" + str(n_img_2).zfill(3)
        rgb_flow = flow2rgb(div_flow*flow_output, max_value=None)
        to_save_rgb = (rgb_flow * 255).astype(np.uint8).transpose(1,2,0)
        imwrite(file_name + '.png', to_save_rgb)

        to_save_np = (div_flow*flow_output).cpu().numpy().transpose(1,2,0)
        np.save(file_name + '.npy', to_save_np)



In [3]:
device = set_cuda()
model, div_flow = load_model(path_to_model, device)

inference(1,50, "swan", device, model, div_flow)


cpu
=> using pre-trained model 'flownets'


C:\Users\Alex\AppData\Local\Temp\ipykernel_15920\196282575.py:61: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  img1 = imread(path_img1)
C:\Users\Alex\AppData\Local\Temp\ipykernel_15920\196282575.py:62: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  img2 = imread(path_img2)
